In [1]:
from tesseraclassifier import habitat_classifier
from tesseraclassifier import classifier_utils
import geopandas as gpd
import tempfile

import importlib
importlib.reload(classifier_utils)
importlib.reload(habitat_classifier)


<module 'tesseraclassifier.habitat_classifier' from '/home/jk871/tessera-classification/src/tesseraclassifier/habitat_classifier.py'>

In [2]:
roi_name = "Cairngorms"
dir_path = "/home/jk871/cci_workshop"

shapefile_path = f"../workshop_shapefiles/SG_CairngormsNationalPark_2010.shp"
entire_roi = gpd.read_file(shapefile_path)

labels_json_file = f'../workshop_labels/cairngorms_labels_new.json'
labels = classifier_utils.load_training_labels_from_json(labels_json_file)

output_dir = f"{dir_path}/classification_outputs/{roi_name}"

In [4]:
hc = habitat_classifier.HabitatClassifier(labels, 2024)


--- Step 1: Preparing Training Data ---
Fetching list of all available tiles...
Found 146452 total available tiles             for 2024.

Processing point 1/119 at                     (57.2032, -3.5610)...
  > Match found! Point belongs to tile (57.25, -3.55).

Processing point 2/119 at                     (57.1343, -3.6756)...
  > Match found! Point belongs to tile (57.15, -3.65).

Processing point 3/119 at                     (57.1348, -3.6781)...
  > Match found! Point belongs to tile (57.15, -3.65).

Processing point 4/119 at                     (57.1880, -3.5335)...
  > Match found! Point belongs to tile (57.15, -3.55).

Processing point 5/119 at                     (57.1885, -3.5357)...
  > Match found! Point belongs to tile (57.15, -3.55).

Processing point 6/119 at                     (57.1960, -3.5401)...
  > Match found! Point belongs to tile (57.15, -3.55).

Processing point 7/119 at                     (57.2498, -3.7296)...
  > Match found! Point belongs to tile (57.25, -3

In [5]:
hc.train()


--- Step 2: Training Model ---
Training model on 112 samples...
✅ Model training complete.


In [7]:
# Classification dry run - just to estimate the size of each ROI
hc.classify_tiles(None, hc.training_tiles, max_workers=8, dry_run=True)
hc.classify_region(None, classifier_utils.roi_from_points(labels), max_workers=8, dry_run=True)
hc.classify_region(None, entire_roi, max_workers=8, dry_run=True)

Found 14 tiles to process.
Found 66 tiles to process.
Found 95 tiles to process.


In [11]:
key = "training_tiles"
temp_dir = tempfile.mkdtemp(prefix=f"tessera_classify_{roi_name}_{key}_")
hc.classify_tiles(temp_dir, hc.training_tiles, max_workers=8)
print(temp_dir)

Found 14 tiles to process.

Starting parallel classification using up to 8 CPU cores...

Parallel processing complete. 14 of 14 tiles were successfully classified.
Intermediate classified tiles saved in: /tmp/tessera_classify_Cairngorms_training_tiles_irxijq05
/tmp/tessera_classify_Cairngorms_training_tiles_irxijq05


In [12]:
hc.mosaic_classification(temp_dir, f"{output_dir}/{key}.tif", cleanup=True)


Stitching 14 tiles into a master mosaic...
✅ Successfully created final classification map: /home/jk871/cci_workshop/classification_outputs/Cairngorms/training_tiles.tif
Cleaning up temporary directory: /tmp/tessera_classify_Cairngorms_training_tiles_irxijq05


In [17]:
key = "bounding_box"
temp_dir = tempfile.mkdtemp(prefix=f"tessera_classify_{roi_name}_{key}_")
hc.classify_region(temp_dir, classifier_utils.roi_from_points(labels), max_workers=30)
print(temp_dir)

Found 66 tiles to process.

Starting parallel classification using up to 30 CPU cores...

Parallel processing complete. 66 of 66 tiles were successfully classified.
Intermediate classified tiles saved in: /tmp/tessera_classify_Cairngorms_bounding_box_boz9i7fu
/tmp/tessera_classify_Cairngorms_bounding_box_boz9i7fu


In [16]:
temp_dir

'/tmp/tessera_classify_Cairngorms_bounding_box_ixzsdab5'

In [18]:
hc.mosaic_classification(temp_dir, f"{output_dir}/{key}.tif", cleanup=True)


Stitching 66 tiles into a master mosaic...
✅ Successfully created final classification map: /home/jk871/cci_workshop/classification_outputs/Cairngorms/bounding_box.tif
Cleaning up temporary directory: /tmp/tessera_classify_Cairngorms_bounding_box_boz9i7fu


In [19]:
key = "entire_roi"
temp_dir = tempfile.mkdtemp(prefix=f"tessera_classify_{roi_name}_{key}_")
hc.classify_region(temp_dir, entire_roi, max_workers=50)
print(temp_dir)

Found 95 tiles to process.

Starting parallel classification using up to 50 CPU cores...



Parallel processing complete. 95 of 95 tiles were successfully classified.
Intermediate classified tiles saved in: /tmp/tessera_classify_Cairngorms_entire_roi_48s9h47m
/tmp/tessera_classify_Cairngorms_entire_roi_48s9h47m


In [20]:
hc.mosaic_classification(temp_dir, f"{output_dir}/{key}.tif", cleanup=True)


Stitching 95 tiles into a master mosaic...
✅ Successfully created final classification map: /home/jk871/cci_workshop/classification_outputs/Cairngorms/entire_roi.tif
Cleaning up temporary directory: /tmp/tessera_classify_Cairngorms_entire_roi_48s9h47m
